## Abstract
Food insecurity remains a critical issue in the United States, affecting millions of households across diverse demographics. This analysis explores patterns of food insecurity using national survey data, focusing on geographic distribution, demographic disparities, and its correlation with poverty. The visualizations presented aim to challenge the perception that hunger is an “elsewhere” problem and demonstrate its prevalence within U.S. borders. The findings highlight vulnerable populations and regional trends, setting the stage for informed policy discussions.

## Introduction
The question guiding this analysis is: *How widespread is food insecurity in the United States, and which populations are most affected?* The objective is to uncover patterns that reveal the depth of the problem and its relationship to poverty, while considering demographic and regional factors. This story is constructed through a series of visualizations that collectively illustrate the scope and complexity of food insecurity.

## Data Preparation
The dataset used for this analysis originates from the U.S. Current Population Survey (CPS) and represents data collected in the year 2023. It was obtained from the official government data catalog at https://catalog.data.gov/dataset/?tags=food-insecurity. The raw file contains detailed household and individual-level information, including demographic attributes, income levels, and food security indicators. For this study, relevant variables were selected to measure food insecurity status, poverty classification, age, gender, and geographic location. Weighted calculations were applied using survey weights to ensure nationally representative estimates. Age groups were categorized into children, adults, and seniors, and gender was classified based on available binary categories. State-level identifiers were mapped from FIPS codes to state names and abbreviations to enable geographic visualization. These preparation steps ensured that the data was structured for meaningful analysis and accurate representation of food insecurity patterns across the United States.

In [1]:
# imports
import pandas as pd
import numpy as np
import plotly.express as px
import geopandas as gpd


In [2]:
# Adjust chunksize if memory is limited
df = pd.read_csv('/content/sample_data/dec23pub.csv', low_memory=False)

# Preview columns
print(df.columns)

Index(['HRHHID', 'HRMONTH', 'HRYEAR4', 'HURESPLI', 'HUFINAL', 'FILLER',
       'HETENURE', 'HEHOUSUT', 'HETELHHD', 'HETELAVL',
       ...
       'HRFS30D1', 'HRFS30D2', 'HRFS30D3', 'HRFS30D4', 'HRFS30D5', 'HRFS30D6',
       'HRFS30D7', 'HRFS30D8', 'HRFS30D9', 'HRFS30DE'],
      dtype='object', length=508)


In [3]:
# Select Relevant Columns
cols = ['GESTFIPS', 'PESEX', 'PRTAGE', 'HRPOOR', 'HEFAMINC', 'HRFS12CX', 'HWHHWGT']
data = df[cols].copy()

# Map FIPS to state names
fips_map = {
    1:'Alabama',2:'Alaska',4:'Arizona',5:'Arkansas',6:'California',8:'Colorado',9:'Connecticut',
    10:'Delaware',11:'District of Columbia',12:'Florida',13:'Georgia',15:'Hawaii',16:'Idaho',
    17:'Illinois',18:'Indiana',19:'Iowa',20:'Kansas',21:'Kentucky',22:'Louisiana',23:'Maine',
    24:'Maryland',25:'Massachusetts',26:'Michigan',27:'Minnesota',28:'Mississippi',29:'Missouri',
    30:'Montana',31:'Nebraska',32:'Nevada',33:'New Hampshire',34:'New Jersey',35:'New Mexico',
    36:'New York',37:'North Carolina',38:'North Dakota',39:'Ohio',40:'Oklahoma',41:'Oregon',
    42:'Pennsylvania',44:'Rhode Island',45:'South Carolina',46:'South Dakota',47:'Tennessee',
    48:'Texas',49:'Utah',50:'Vermont',51:'Virginia',53:'Washington',54:'West Virginia',
    55:'Wisconsin',56:'Wyoming'
}
data['State'] = data['GESTFIPS'].map(fips_map)

# Age groups
bins = [0,18,65,120]
labels = ['Children','Adults','Seniors']
data['AgeGroup'] = pd.cut(data['PRTAGE'], bins=bins, labels=labels, right=False)

# Food insecurity flag (HRFS12CX: 1=food secure, 2+=insecure)
data['FoodInsecure'] = np.where(data['HRFS12CX']>=2,1,0)

print(data)

        GESTFIPS  PESEX  PRTAGE  HRPOOR  HEFAMINC  HRFS12CX   HWHHWGT  \
0              1      2      43       2        14         1  26465793   
1              1      1      44       2        14         1  26465793   
2              1      2      13       2        14         1  26465793   
3              1      2      15       2        14         1  26465793   
4              1      2      20       2        14         1  26465793   
...          ...    ...     ...     ...       ...       ...       ...   
126827        56      2      64      -1        12        -1   2499573   
126828        56      1      35      -1        13        -1   3092193   
126829        56      2      31      -1        13        -1   3092193   
126830        56      2       0      -1        13        -1   3092193   
126831        56     -1      -1      -1        -1        -1         0   

          State  AgeGroup  FoodInsecure  
0       Alabama    Adults             0  
1       Alabama    Adults             0

In [4]:
# Weighted food insecurity by state
state_agg = data.groupby('State').apply(
    lambda x: np.average(x['FoodInsecure'], weights=x['HWHHWGT']),
    include_groups=False
).reset_index(name='FoodInsecurityRate')

# Gender & Age breakdown
gender_age = data.groupby(['PESEX','AgeGroup']).apply(
    lambda x: np.average(x['FoodInsecure'], weights=x['HWHHWGT']),
    include_groups=False
).reset_index(name='Rate')
gender_age['Gender'] = gender_age['PESEX'].map({1:'Male',2:'Female'})

#
poverty_state = data.groupby('State').apply(
    lambda x: np.average(x['HRPOOR']==1, weights=x['HWHHWGT']),
    include_groups=False
).reset_index(name='PovertyRate')
poverty_merge = state_agg.merge(poverty_state, on='State')

# Age progression
age_progression = data.groupby('AgeGroup').apply(
    lambda x: np.average(x['FoodInsecure'], weights=x['HWHHWGT']),
    include_groups=False
).reset_index(name='Rate')

/tmp/ipython-input-3388478313.py:8: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  gender_age = data.groupby(['PESEX','AgeGroup']).apply(
/tmp/ipython-input-3388478313.py:22: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  age_progression = data.groupby('AgeGroup').apply(


### Visualization 1: Food Insecurity Rate by State
A choropleth map displays food insecurity rates across U.S. states. Each state is shaded according to its weighted food insecurity percentage, revealing geographic disparities. States in the South and certain rural areas exhibit higher rates, emphasizing that food insecurity is not confined to isolated pockets but is a nationwide concern. This visualization establishes the foundation of the story by showing where the problem is most severe.

In [23]:
# Add state abbreviations
state_abbrev = {
    'Alabama':'AL','Alaska':'AK','Arizona':'AZ','Arkansas':'AR','California':'CA','Colorado':'CO','Connecticut':'CT',
    'Delaware':'DE','District of Columbia':'DC','Florida':'FL','Georgia':'GA','Hawaii':'HI','Idaho':'ID',
    'Illinois':'IL','Indiana':'IN','Iowa':'IA','Kansas':'KS','Kentucky':'KY','Louisiana':'LA','Maine':'ME',
    'Maryland':'MD','Massachusetts':'MA','Michigan':'MI','Minnesota':'MN','Mississippi':'MS','Missouri':'MO',
    'Montana':'MT','Nebraska':'NE','Nevada':'NV','New Hampshire':'NH','New Jersey':'NJ','New Mexico':'NM',
    'New York':'NY','North Carolina':'NC','North Dakota':'ND','Ohio':'OH','Oklahoma':'OK','Oregon':'OR',
    'Pennsylvania':'PA','Rhode Island':'RI','South Carolina':'SC','South Dakota':'SD','Tennessee':'TN',
    'Texas':'TX','Utah':'UT','Vermont':'VT','Virginia':'VA','Washington':'WA','West Virginia':'WV',
    'Wisconsin':'WI','Wyoming':'WY'
}

state_agg['StateCode'] = state_agg['State'].map(state_abbrev)

fig = px.choropleth(
    state_agg,
    locations='StateCode',
    locationmode='USA-states',
    color='FoodInsecurityRate',
    scope='usa',
    color_continuous_scale='Reds',
    title='The United States Food Insecurity Rates Per States'
)


fig.show()

### Visualization 2: Food Insecurity Across Age Groups
A line chart illustrates food insecurity percentages for three age categories: children, adults, and seniors. The chart shows that children experience the highest vulnerability, suggesting long-term implications for health and development. This visualization adds a temporal dimension to the story, indicating that early-life insecurity may have lasting effects.

In [24]:
# Convert Rate to percentage
age_progression['RatePct'] = age_progression['Rate'] * 100

fig = px.line(
    age_progression,
    x='AgeGroup',
    y='RatePct',
    markers=True,
    title='Children Face the Highest Food Insecurity Risk',
    labels={'RatePct':'Food Insecurity (%)'}
)
fig.update_traces(line=dict(color='firebrick', width=4), marker=dict(size=12, color='darkred'))
fig.update_layout(yaxis=dict(ticksuffix='%'), plot_bgcolor='white')
fig.add_annotation(
    x='Children', y=age_progression.loc[age_progression['AgeGroup']=='Children','RatePct'].values[0],
    text='Starts early: Children most vulnerable',
    showarrow=True, arrowhead=2, ax=40, ay=-40
)
fig.show()

### Visualization 3: Food Insecurity by Gender and Age Group
A grouped bar chart compares food insecurity rates between males and females across age groups. Women consistently show higher rates than men, particularly among adults and seniors. This visualization highlights gender-based disparities and identifies populations that may require targeted interventions.

In [7]:
# Convert to percentage
gender_age['RatePct'] = gender_age['Rate'] * 100

fig = px.bar(
    gender_age,
    x='AgeGroup',
    y='RatePct',
    color='Gender',
    barmode='group',
    title='Women Experience Higher Food Insecurity Across All Ages',
    labels={'RatePct':'Food Insecurity (%)'},
    color_discrete_map={'Male':'steelblue','Female':'darkred'}
)
fig.update_layout(yaxis=dict(ticksuffix='%'), plot_bgcolor='white')
fig.add_annotation(
    text="Note: CPS data only includes binary gender (Male/Female)",
    xref="paper", yref="paper",
    x=0, y=-0.2,
    showarrow=False,
    font=dict(size=12, color="gray")
)
fig.show()


### Visualization 4: Poverty vs. Food Insecurity by State
A scatter plot examines the relationship between poverty rates and food insecurity across states, with points colored by region. A clear positive trend emerges, indicating that states with higher poverty rates tend to have higher food insecurity. This visualization reinforces the economic dimension of the issue, linking food insecurity to structural poverty.

In [8]:
# Assign regions to states
regions = {
    'Alabama':'South','Alaska':'West','Arizona':'West','Arkansas':'South','California':'West',
    'Colorado':'West','Connecticut':'Northeast','Delaware':'South','District of Columbia':'South',
    'Florida':'South','Georgia':'South','Hawaii':'West','Idaho':'West','Illinois':'Midwest',
    'Indiana':'Midwest','Iowa':'Midwest','Kansas':'Midwest','Kentucky':'South','Louisiana':'South',
    'Maine':'Northeast','Maryland':'South','Massachusetts':'Northeast','Michigan':'Midwest',
    'Minnesota':'Midwest','Mississippi':'South','Missouri':'Midwest','Montana':'West','Nebraska':'Midwest',
    'Nevada':'West','New Hampshire':'Northeast','New Jersey':'Northeast','New Mexico':'West',
    'New York':'Northeast','North Carolina':'South','North Dakota':'Midwest','Ohio':'Midwest',
    'Oklahoma':'South','Oregon':'West','Pennsylvania':'Northeast','Rhode Island':'Northeast',
    'South Carolina':'South','South Dakota':'Midwest','Tennessee':'South','Texas':'South',
    'Utah':'West','Vermont':'Northeast','Virginia':'South','Washington':'West','West Virginia':'South',
    'Wisconsin':'Midwest','Wyoming':'West'
}

poverty_merge['Region'] = poverty_merge['State'].map(regions)

fig = px.scatter(
    poverty_merge,
    x='PovertyRate',
    y='FoodInsecurityRate',
    color='Region',
    hover_name='State',
    trendline='ols',
    title='Higher Poverty Strongly Correlates with Food Insecurity',
    labels={'PovertyRate':'Poverty Rate (%)','FoodInsecurityRate':'Food Insecurity Rate (%)'},
    color_discrete_map={'South':'firebrick','West':'steelblue','Midwest':'darkgreen','Northeast':'purple'}
)
fig.update_traces(marker=dict(size=12, opacity=0.8, line=dict(width=1, color='black')))
fig.update_layout(xaxis=dict(ticksuffix='%'), yaxis=dict(ticksuffix='%'), plot_bgcolor='white')
fig.add_annotation(
    text="Southern states cluster at high poverty & insecurity",
    xref="paper", yref="paper",
    x=0.5, y=-0.15,
    showarrow=False,
    font=dict(size=14, color="firebrick")
)
fig.show()

## Conclusion
The analysis demonstrates that food insecurity in the United States is both widespread and deeply interconnected with socioeconomic and demographic factors. The choropleth map reveals significant regional disparities, with **Southern** states and parts of the **Midwest** experiencing the highest rates of food insecurity. The age-based line chart shows that children are the most vulnerable group, with food insecurity rates exceeding 22%, indicating that the problem begins early in life and may have long-term consequences. Gender-based comparisons highlight that women consistently face higher food insecurity than men across all age groups, underscoring the role of gender in vulnerability. Finally, the scatter plot establishes a strong positive correlation between poverty and food insecurity, confirming that economic hardship is a primary driver of hunger.
The question posed — *How widespread is food insecurity in the United States, and which populations are most affected?* — is answered through these findings: food insecurity is not an isolated issue but a systemic challenge affecting millions, with disproportionate impact on children, women, and residents of economically disadvantaged states. These insights emphasize the urgency of addressing food insecurity as a national priority rather than viewing it as a distant or localized problem.